## Input layer 바꾸기 -> patch embedding to linear

In [9]:
!pip install easydict

You should consider upgrading via the 'c:\users\quhb2\anaconda3\envs\torch-1.9\python.exe -m pip install --upgrade pip' command.


In [16]:
from easydict import EasyDict
from torchsummary import summary

In [70]:
import timm 

model = timm.create_model('vit_base_patch16_224', pretrained=True)

In [71]:
data = torch.randn([3,224,224])
# data.shape
summary(model, (3,224,224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 768, 14, 14]         590,592
          Identity-2             [-1, 196, 768]               0
        PatchEmbed-3             [-1, 196, 768]               0
           Dropout-4             [-1, 197, 768]               0
         LayerNorm-5             [-1, 197, 768]           1,536
            Linear-6            [-1, 197, 2304]       1,771,776
           Dropout-7         [-1, 12, 197, 197]               0
            Linear-8             [-1, 197, 768]         590,592
           Dropout-9             [-1, 197, 768]               0
        Attention-10             [-1, 197, 768]               0
         Identity-11             [-1, 197, 768]               0
        LayerNorm-12             [-1, 197, 768]           1,536
           Linear-13            [-1, 197, 3072]       2,362,368
             GELU-14            [-1, 19

In [72]:
model

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn

In [73]:
model.blocks

Sequential(
  (0): Block(
    (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
    (attn): Attention(
      (qkv): Linear(in_features=768, out_features=2304, bias=True)
      (attn_drop): Dropout(p=0.0, inplace=False)
      (proj): Linear(in_features=768, out_features=768, bias=True)
      (proj_drop): Dropout(p=0.0, inplace=False)
    )
    (drop_path): Identity()
    (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
    (mlp): Mlp(
      (fc1): Linear(in_features=768, out_features=3072, bias=True)
      (act): GELU()
      (fc2): Linear(in_features=3072, out_features=768, bias=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
  )
  (1): Block(
    (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
    (attn): Attention(
      (qkv): Linear(in_features=768, out_features=2304, bias=True)
      (attn_drop): Dropout(p=0.0, inplace=False)
      (proj): Linear(in_features=768, out_features=768, bias=True)
      (proj_drop): Dropout(p=0.0, inplac

In [82]:
import torch
import torch.nn as nn

model = nn.Sequential(nn.Linear(768, 1024), model.blocks)
model

AttributeError: 'Sequential' object has no attribute 'blocks'

In [84]:
summary(model, (768, 256), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1             [-1, 768, 768]         197,376
         LayerNorm-2             [-1, 768, 768]           1,536
            Linear-3            [-1, 768, 2304]       1,771,776
           Dropout-4         [-1, 12, 768, 768]               0
            Linear-5             [-1, 768, 768]         590,592
           Dropout-6             [-1, 768, 768]               0
         Attention-7             [-1, 768, 768]               0
          Identity-8             [-1, 768, 768]               0
         LayerNorm-9             [-1, 768, 768]           1,536
           Linear-10            [-1, 768, 3072]       2,362,368
             GELU-11            [-1, 768, 3072]               0
          Dropout-12            [-1, 768, 3072]               0
           Linear-13             [-1, 768, 768]       2,360,064
          Dropout-14             [-1, 7